## Install Dependencies and Import Libraries

In [1]:
!pip install tensorflow
!pip install keras
!pip install gym
!pip install gymnasium
!pip install keras-rl2
!pip install stable-baselines3
!pip install traci

ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow


  Using cached keras_rl2-1.0.5-py3-none-any.whl.metadata (304 bytes)
INFO: pip is looking at multiple versions of keras-rl2 to determine which version is compatible with other requirements. This could take a while.
  Using cached keras_rl2-1.0.4-py3-none-any.whl.metadata (353 bytes)
  Using cached keras-rl2-1.0.3.tar.gz (40 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached keras-rl2-1.0.2.tar.gz (40 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject

ERROR: Cannot install keras-rl2==1.0.0, keras-rl2==1.0.1, keras-rl2==1.0.2, keras-rl2==1.0.3, keras-rl2==1.0.4 and keras-rl2==1.0.5 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [2]:
import numpy as np
import random
import gymnasium
import sumolib
import time
import traci

## OpenAI Gymnasium Environment

In [3]:
class SumoEnv(gymnasium.Env):
  def __init__(self, use_gui=False, use_random=False, use_actions=True, spawn_rate=0.5):
    super().__init__() # Initializes the parent class

    # Check if TraCI is already loaded; if so, close it
    if traci.isLoaded():
      traci.close()

    # Define the Discrete action space with gymnasium.spaces.Discrete(n)
    # choices are up & down = green, or l & r = green
    self.action_space = gymnasium.spaces.Discrete(3)

    # Define the Box observation space with gymnasium.spaces.Box()
    # Note the structure of the Box parameters requires NumPy arrays!
    # Define lanes directly connected to the intersection
    self.lanes = {
      # Edge 1 (West to East)
      "E0_2": {"type": [1, 0, 0], "phases": [4]}, # Left Lane
      "E0_1": {"type": [0, 1, 0], "phases": [2]}, # Straight Lane
      "E0_0": {"type": [0, 0, 1], "phases": [2]}, # Right Lane

      # Edge 2 (East to West)
      "E9_0": {"type": [0, 1, 1], "phases": [2]}, # Straight and right lane
      "E9_1": {"type": [1, 0, 0], "phases": [4]}, # left lane

      # Edge 3 (South to North)
      "E7_2": {"type": [0, 0, 1], "phases": [0]}, # Right lane
      "E7_1": {"type": [0, 1, 0], "phases": [0]}, # Straight lane
      "E7_0": {"type": [1, 0, 0], "phases": [0]}, # Left lane

      # Edge 4 (North to South)
      "E3_3": {"type": [1, 0, 0], "phases": [0]}, # Left lane
      "E3_2": {"type": [0, 1, 0], "phases": [0]}, # Straight lane
      "E3_1": {"type": [0, 1, 0], "phases": [0]}, # Straight lane
      "E3_0": {"type": [0, 0, 1], "phases": [0]}, # Right lane      
    }
    
    self.last_phase_change_time = { # Each key corresponds to each of the end state phases
      0: 0,
      2: 0,
      4: 0
    }
    self.vehicle_emissions = {}
    self.max_wait_time = 1000
    num_lanes = 12
    num_metrics_per_lane = 5
    num_lane_type_features = 3 # [left, straight, right] "one-hot" encoding
    observation_size = 2 + num_lanes * (num_metrics_per_lane + num_lane_type_features)

    max_cars = 100 # CHANGE FOR ACTUAL MAX. NUMBER OF CARS
    self.max_cars = max_cars
    
    self.car_spawn_rate = spawn_rate # cars spawn at 40% chance

    # np array structure: [traffic_light_phase][positions][speeds], dtype=np.float32
    self.observation_space = gymnasium.spaces.Box(
      low=np.array([0.0] * observation_size),
      high=np.array([1.0] * observation_size), # MUST CHANGE THE TRAFFIC LIGHT PHASE VALUE TO THE RESPECTIVE MAX!
      dtype=np.float32
    )
    
    # Upon each render of the SumoEnv Class, we should start the simulation
    # Implement the sumo_binary, sumo_config, and traci.start from test_demo.py
    self.use_gui = use_gui
    if use_gui:
      sumo_binary = sumolib.checkBinary('sumo-gui')
    else:
      sumo_binary = sumolib.checkBinary('sumo')

    # use the proper .sumocfg file depending on if want predefined or random cars
    self.use_random = use_random
    if use_random:
      sumo_config = "./network/western.sumocfg"
    else:
      sumo_config = "./network/western.sumocfg"
    
    # used to track number of deployed cars
    if self.use_random:
      self.deployed_counter = 0
    else:
      self.deployed_counter = 1

    # store these variables for use later
    self.sumo_binary = sumo_binary
    self.sumo_config = sumo_config

    # Start the simulation
    self.started = False

    # Track cumulative metrics
    self.vehicle_wait_log = {}
    self.total_congestion_log = []
    self.total_speed_log = []

    # use actions argument: False = use the timer based system instead (for comparing between agent and real-life)
    self.use_actions = use_actions

    # Define consistent pause time for sumo-gui visualization
    self.pause_time = 0.35

  def step(self, action):
    # On first step, start the traci sim
    if not self.started:
      traci.start([self.sumo_binary, "--start", "-c", self.sumo_config])
      self.started = True
      traffic_light_id = traci.trafficlight.getIDList()[0]
      traci.trafficlight.setPhase(traffic_light_id, 0) # Ensure light phases are all manually controlled

      if self.use_actions:
        traci.trafficlight.setPhaseDuration(traffic_light_id, 99999)  # Hold this phase indefinitely
    
    # Perform the action
    if self.use_actions:
      self.perform_action(action)
    
    # Spawn in a car if suits the spawn rate on step
    if self.use_random:
      if (random.random() < self.car_spawn_rate) and (self.deployed_counter < self.max_cars):
        self.spawn_random_car(self.deployed_counter)
        self.deployed_counter += 1
  
    # get the most updated vehicle emission for each vehicle in the simulation
    for vehicle_id in traci.vehicle.getIDList():
      if vehicle_id not in self.vehicle_emissions:
        self.vehicle_emissions[vehicle_id] = 0
      self.vehicle_emissions[vehicle_id] = traci.vehicle.getCO2Emission(vehicle_id)

    # Advance the simulation by one step
    traci.simulationStep()
    if self.use_gui: # pause in between steps to slow down if in 'simulation mode'
      time.sleep(self.pause_time) 
    # print("Step: " + str(traci.simulation.getTime()))
    # Get the new state
    observation = self.get_state()

    # Calculate the reward
    reward = self.calculate_reward()

    # Determine if simulation is done
    done = self.is_done()

    # Set placeholder for info -> returns the tracked cumulative metrics IF done
    info = {
      "vehicle_wait_log": self.vehicle_wait_log if done else None,
      "total_congestion_avg": (sum(self.total_congestion_log) / len(self.total_congestion_log)) if done and self.total_congestion_log else None,
      "total_speed_avg": (sum(self.total_speed_log) / len(self.total_speed_log)) if done and self.total_speed_log else None,
      "emissions": self.calculate_mean_emission()
    }

    # Set placeholder for truncated
    truncated = False

    # set 'observation' to a numpy array
    observation = np.array(observation, dtype=np.float32)

    # Return step information (MUST follow this order of variables!!!)
    return observation, reward, done, truncated, info

  def render(self):
    # render needs to exist in the Gymnasium env, as it is an essential aspect
    # however we might not need to put anything inside it, hence 'pass'
    # this depends on if the command -> traci.simulationStep() exists somewhere else in the Class
    pass

  def reset(self, seed=None, options=None):

    # resets the gymnasium.Env parent class
    super().reset(seed=seed)

    # close the simulation (reset)
    if not self.use_gui: # traci.load() doesn't work for sumo-gui - i.e. can only run once
      traci.load(["-c", self.sumo_config])

    # reset counter variables
    if self.use_random:
      self.deployed_counter = 0
    else:
      self.deployed_counter = 1

    # convert 'observation' to a NumPy array
    observation = np.array(self.get_state(), dtype=np.float32)

    self.last_phase_change_time = { # note: each of the keys correspond to one of my end state phases
      0: 0,
      2: 0,
      4: 0,
    }

    self.vehicle_emissions = {}
    self.vehicle_wait_log = {}
    self.total_congestion_log = []
    self.total_speed_log = []

    # return 'observation' and 'info' --> MUST be in this form
    return observation, {}

  def get_state(self):
    # Define a speed threshold for "stopped" vehicles
    speed_threshold = 0.1  # Vehicles with speed < 0.1 m/s are considered stopped

    state = []

    # 1. Traffic light phase (normalized to [0, 1])
    traffic_light_phase = traci.trafficlight.getPhase(traci.trafficlight.getIDList()[0])
    state.append(traffic_light_phase / 5.0)  # Normalize phase to [0, 1]
    for phase_change_time in self.last_phase_change_time:
      if phase_change_time == traffic_light_phase:
        self.last_phase_change_time[phase_change_time] = 0
      else:
        self.last_phase_change_time[phase_change_time] += 1

    # 2. Time since last phase change (normalized to [0, 1])
    current_time = traci.simulation.getTime()
    time_since_last_change = current_time - min(self.last_phase_change_time.values())
    state.append(time_since_last_change / self.max_wait_time)  # Normalize using max_wait_time

    # 3. Per-lane metrics (only for lanes directly connected to the intersection)
    for lane_id, lane_info in self.lanes.items():
        vehicle_ids = traci.lane.getLastStepVehicleIDs(lane_id)

        # Metric 1: Number of vehicles (normalized to [0, 1])
        num_vehicles = len(vehicle_ids)
        state.append(num_vehicles / self.max_cars)

        # Metric 2: Queue length (number of vehicles with speed < threshold, normalized to [0, 1])
        queue_length = sum(1 for v_id in vehicle_ids if traci.vehicle.getSpeed(v_id) < speed_threshold)
        state.append(queue_length / self.max_cars)

        # Metric 3: Total wait time (normalized to [0, 1])
        total_wait_time = sum(traci.vehicle.getWaitingTime(v_id) for v_id in vehicle_ids)
        state.append(total_wait_time / self.max_wait_time)  # Normalize using max_wait_time

        # Metric 4: Average speed (normalized to [0, 1])
        avg_speed = np.mean([traci.vehicle.getSpeed(v_id) for v_id in vehicle_ids]) if vehicle_ids else 0
        state.append(avg_speed / traci.lane.getMaxSpeed(lane_id))  # Normalize to [0, 1]

        # Metric 5: Time since last visited  
        minn = float("inf")
        for phase in lane_info["phases"]:
          if self.last_phase_change_time[phase] < minn:
            minn = self.last_phase_change_time[phase]
        state.append(minn / float(max(self.last_phase_change_time.values())))

        # Lane type (one-hot encoded)
        lane_type = lane_info["type"]
        state.extend(lane_type)

    return np.array(state, dtype=np.float32)

  def perform_action(self, action):

    light_id = traci.trafficlight.getIDList()[0]
    current_phase = traci.trafficlight.getPhase(light_id)

    """
    Phases: 
      (0) E & W = red, N & S = green; no left turn signal available, so cars waiting for N & S left turn makes the turn when it is safe to do so
      (1) E & W = red, N & S = yellow; no left turn signal available; cars waiting for N & S left turn must agree that this is a yellow light for them as well
      (2) E & W = green, N & S = red; left turn signal is still not green. So, it is a good choice for E & W left turners to wait.
      (3) E & W = yellow, N & S = red; left turn signal is still not green. 
      (4) E & W = red, N & S = red, E & W LEFT TURN = green;
      (5) E & W = red, N & S = red, E & W LEFT TURN = yellow;

      Define action 0 as switching green to N & S ALL; 
      Define action 1 as switching green to E & W ALL; 
      Define action 2 as switching green to E & W left turn advance; 

    """

    if action == 0 and current_phase != 0:
      if current_phase == 2:
        traci.trafficlight.setPhase(light_id, 3) # traffic light transitions to yellow in the very next phase
      elif current_phase == 4:
        traci.trafficlight.setPhase(light_id, 5) # traffic light transitions to yellow in the very next phase
      traci.trafficlight.setPhaseDuration(light_id, 3) # ensure the traffic light stays in any state for at least 3 seconds
      self.skip_steps(3) # ensure the light is green for at least 3 consecutive seconds
      traci.trafficlight.setPhase(light_id, 0) # set N & S green
      traci.trafficlight.setPhaseDuration(light_id, 99999) # Hold phase 0 indefinitely
      self.skip_steps(5) # ensure the light is green for at least 3 consecutive seconds
      # print("Set to phase 0") # print a confirmation message indicating the traffic light is set to phase 0

    elif action == 1 and current_phase != 2:
      if current_phase == 0:
        traci.trafficlight.setPhase(light_id, 1) # transition to yellow in next phase
      elif current_phase == 4:
        traci.trafficlight.setPhase(light_id, 5) # transition to yellow in next phase
      traci.trafficlight.setPhaseDuration(light_id, 3) # traffic light stays in any state for at least 3 seconds
      self.skip_steps(3) # ensure the light is green for at least 3 seconds
      traci.trafficlight.setPhase(light_id, 2) # set E & W green
      traci.trafficlight.setPhaseDuration(light_id, 99999) # Hold phase 2 indefinitely
      self.skip_steps(5) # ensure the light is green for at least 3 consecutive seconds
      # print("Set to phase 2") # print a confirmation message indicating the traffic light is set to phase 2      

    elif action == 2 and current_phase != 4:
      if current_phase == 0:
        traci.trafficlight.setPhase(light_id, 1) # transition to yellow in next phase
      elif current_phase == 2:
        traci.trafficlight.setPhase(light_id, 3) # transition to yellow in next phase
      traci.trafficlight.setPhaseDuration(light_id, 3) # traffic light stays in any state for at least 3 seconds
      self.skip_steps(3) # ensure the light is green for at least 3 seconds
      traci.trafficlight.setPhase(light_id, 4) # set E & W LEFT TURN green
      traci.trafficlight.setPhaseDuration(light_id, 99999) # Hold phase 4 indefinitely
      self.skip_steps(5) # ensure the light is green for at least 3 consecutive seconds
      # print("Set to phase 4") # print a confirmation message indicating the traffic light is set to phase 4


  def calculate_reward(self):
    # REWARD FUNCTION: Calculate the reward (should be negative if in a poor state i.e. high congestion)
    lane_ids = self.lanes.keys()
    vehicle_ids = traci.vehicle.getIDList()
    light_id = traci.trafficlight.getIDList()[0]
    current_phase = traci.trafficlight.getPhase(light_id)
    reward = 0
    try:

      for lane in self.lanes.keys():
        if current_phase in self.lanes[lane]["phases"]:
          reward += len(traci.lane.getLastStepVehicleIDs(lane))

        else:
          cur_wait_time = traci.lane.getWaitingTime(lane)
          cur_phases = self.lanes[lane]["phases"]
          min_phase = cur_phases[0]
          for j in range(1, len(cur_phases)):
            if self.last_phase_change_time[cur_phases[j]] < self.last_phase_change_time[min_phase]:
              min_phase = cur_phases[j]
          reward -= cur_wait_time * (self.last_phase_change_time[min_phase] / max(self.last_phase_change_time.values()))

      congestion = self.calculate_congestion(vehicle_ids)
      wait_time = self.calculate_avg_wait_time(lane_ids)
      stops = self.calculate_total_stops(lane_ids)
      avg_speed = self.calculate_avg_speed(vehicle_ids)# -> would be maximize so don't multiply by -1
      #reward = -1*(0.5*congestion + 0.8*wait_time + stops) + 0.75*avg_speed # minimize all terms

    except:

      print("BIG NONO")
      reward = 0

    return reward

  def skip_steps(self, x):
    for _ in range(x): 
      if self.use_random:
        if (random.random() < self.car_spawn_rate) and (self.deployed_counter < self.max_cars):
          self.spawn_random_car(self.deployed_counter)
          self.deployed_counter += 1

      # Advance the simulation by one step
      traci.simulationStep()
      if self.use_gui: # pause in between stpes to slow down if in 'simulation mode'
        time.sleep(self.pause_time)

  def is_done(self):
    max_time = self.max_wait_time  # Example maximum simulation time
    return (traci.simulation.getTime() >= max_time or len(traci.vehicle.getIDList()) == 0) and self.deployed_counter >= self.max_cars -1


  # METRICS:
  def calculate_congestion(self, vehicle_ids):
    congestion = 0
    current_time = traci.simulation.getTime()  # Get the current simulation time
    
    for vehicle_id in vehicle_ids:
      departure_time = traci.vehicle.getDeparture(vehicle_id)  # Get each vehicle's departure time
      speed = traci.vehicle.getSpeed(vehicle_id)  # Get the vehicle's current speed
      
      # Check if the vehicle is stopped and not just starting/departing
      if speed == 0 and current_time not in range(int(departure_time) - 1, int(departure_time) + 2):
          congestion += 1  # Increment congestion counter for stopped vehicles
  
    # update congestion log
    self.total_congestion_log.append(congestion)

    return congestion

  def calculate_mean_emission(self):
    total_emissions = sum(self.vehicle_emissions.values())
    num_vehicles = len(self.vehicle_emissions)
    try:
        return total_emissions / num_vehicles
    except:
        return 0


  def calculate_avg_wait_time(self, lane_ids):
    wait_times = []
    
    # total wait time of cars in all lanes
    for lane_id in lane_ids:
      # total wait time of all cars in one lane
      for vehicle_id in traci.lane.getLastStepVehicleIDs(lane_id):
          wait_time = traci.vehicle.getWaitingTime(vehicle_id)

          # update the wait log
          if vehicle_id in self.vehicle_wait_log:
            self.vehicle_wait_log[vehicle_id] = max(self.vehicle_wait_log[vehicle_id], wait_time) # only want the greatest average wait time for each vehicle
          else:
            self.vehicle_wait_log[vehicle_id] = wait_time

          wait_times.append(wait_time)
  
    avg_wait_time = sum(wait_times)/len(wait_times) if wait_times else 0
    
    return avg_wait_time

  def calculate_total_stops(self, lane_ids):
    total_stops = 0
    for lane_id in lane_ids:
      stops_in_lane = traci.lane.getLastStepHaltingNumber(lane_id)
      total_stops += stops_in_lane

    return total_stops
    
  def calculate_avg_speed(self, vehicle_ids):
    total_speed = sum(traci.vehicle.getSpeed(v_id) for v_id in vehicle_ids)
    avg_speed = total_speed / len(vehicle_ids) if vehicle_ids else 0

    # update total speed log
    self.total_speed_log.append(avg_speed)

    return avg_speed

  def spawn_random_car(self, step_counter):
    """
    Spawns a random car with a unique ID and assigns it a random route.
    edge_mapping represents the available routes that any car can take; start_edge:end_edge
    """
    edge_mapping = {
        # Key : [value1, value2, value3] = key represents the starting edge for a vehicle : value represents a list of ending edges that vehicles can travel to from the starting edge.
        "E9": ["E2", "E4", "E8"], # from East to West
        "E0": ["E5", "E4", "E8"], # from West to East
        "E7": ["E5", "E4", "E2"], # from South to North
        "E3": ["E8", "E5", "E2"] # from North to South
    }
    vehicle_id = f"rand_car_{step_counter}"

#     edges = traci.edge.getIDList()
#     start_edges = []
#     for edge in edges:
#         if not edge.startswith('-') and not edge.startswith(':'):
#             start_edges.append(edge)
    
    start_edge = random.choice(list(edge_mapping.keys()))
    end_edge = random.choice(edge_mapping[start_edge])

    route_id = f"route_{vehicle_id}"

    try:
        traci.route.add(routeID=route_id, edges=[start_edge, end_edge])

        # Add the vehicle to the simulation
        traci.vehicle.add(vehID=vehicle_id, routeID=route_id)

        # print(f"Deployed random vehicle {vehicle_id} from {start_edge} to {end_edge}")

        # Set a random speed for the vehicle
        traci.vehicle.setSpeed(vehicle_id, random.uniform(5, 15))

    except traci.TraCIException as e:
        pass
        # print(f"Failed to add vehicle {vehicle_id} on route from {start_edge} to {end_edge}: {e}")

## Simulation with SUMO-GUI

In [4]:
'''
    The purpose of this is just to visualize our results!
    Explanation of SumoEnv parameters:
        use_gui = True runs with sumo gui
        use_gui = False runs sumo (all same functionality just doesn't render anything)
        use_random = False tells the env NOT to deploy random vehicles
        use_random = True tells the env to deploy random vehicles
        use_actions = False tells the env that we are running with random actions (note can ONLY use False here since we are using env.action_space.sample() later)
        use_actions = True tell the env that we are running with actions chosen by the agent (will NOT work yet since we don't have an agent!)
'''

env = SumoEnv(use_gui=False, use_random=True, use_actions=True) # use_gui=False sets sumo_binary to 'sumo' instead of 'sumo-gui'

episodes = 1 # note can only be run ONCE with sumo-gui!
score_log = []
wait_log = []
emissions_log = []
congestion_log = []
speed_log = []

for episode in range(1, episodes+1):
    #state, _ = env.reset()
    done = False
    truncated = False
    score = 0
    info = {}

    while not done:
        env.render()
        action = env.action_space.sample()
        state, reward, done, truncated, info = env.step(action)
        score+=reward

    state, _ = env.reset()
    score_log.append(score)

    # Extract wait time metrics
    emissions_log.append(info["emissions"])
    wait_times = info.get("vehicle_wait_log", {}).values()
    total_wait_time = sum(wait_times)
    num_cars = len(wait_times)
    episode_mean_wait = total_wait_time / num_cars if num_cars > 0 else 0
    wait_log.append(episode_mean_wait)

    # Extract congestion and speed metrics
    if info.get("total_congestion_avg") is not None:
        congestion_log.append(info["total_congestion_avg"])
    if info.get("total_speed_avg") is not None:
        speed_log.append(info["total_speed_avg"])

    '''
    score_log.append(score)
    print(info)
    wait_times = info.get('vehicle_wait_log', {}).values()
    total_wait_time = sum(wait_times)
    num_cars = len(wait_times)
    episode_mean_wait = total_wait_time / num_cars
    wait_log.append(episode_mean_wait)
    '''


traci.close()

# Compute and print final metrics
mean_sample_score = np.mean(score_log)
# mean_sample_score = np.mean(score_log)*episodes
mean_wait_time = np.mean(wait_log)
mean_congestion = np.mean(congestion_log) if congestion_log else None
mean_speed = np.mean(speed_log) if speed_log else None
mean_emissions = np.mean(emissions_log) if emissions_log else None

print(f"Mean Score over {episodes} episodes: {mean_sample_score}")
print(f"Mean wait time over {episodes} episodes: {mean_wait_time}")
print(f"Mean congestion over {episodes} episodes: {mean_congestion}")
print(f"Mean speed over {episodes} episodes: {mean_speed}")
print(f"Mean emissions over {episodes} episodes: {mean_emissions} milligrams (mg)")


c:\Users\OWNER\AppData\Local\Programs\Python\Python313\Lib\site-packages\gymnasium\spaces\box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
c:\Users\OWNER\AppData\Local\Programs\Python\Python313\Lib\site-packages\gymnasium\spaces\box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


Mean Score over 1 episodes: -3184.575
Mean wait time over 1 episodes: 10.35
Mean congestion over 1 episodes: 3.186046511627907
Mean speed over 1 episodes: 5.3375270777203925
Mean emissions over 1 episodes: 2632.0184086281715 milligrams (mg)


In [5]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.vec_env import DummyVecEnv

In [6]:
# Create new vec env
env = SumoEnv(use_gui=False, use_random=True, use_actions=True)
env.step(0)
# check_env(env)
env = DummyVecEnv([lambda: env])

c:\Users\OWNER\AppData\Local\Programs\Python\Python313\Lib\site-packages\gymnasium\spaces\box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
c:\Users\OWNER\AppData\Local\Programs\Python\Python313\Lib\site-packages\gymnasium\spaces\box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


In [7]:
# Train Agent
env = SumoEnv(use_gui=False, use_random=True, use_actions=True)
env.step(0)
#check_env(env)
env = DummyVecEnv([lambda: env])
model = PPO("MlpPolicy", env, verbose=1)

spawn_rates = np.arange(0.05, 0.9, 0.05) 

for spawn_rate in spawn_rates:
    env = SumoEnv(use_gui=False, use_random=True, use_actions=True, spawn_rate=spawn_rate)
    env.step(0)
    env = DummyVecEnv([lambda: env])

    model.set_env(env)

    model.learn(total_timesteps=10000)


# can change number of timesteps, however at least 10000 to make a semi-decent one
model.save("./agents/western-agent-various-rates-3") # takes ~30 mins 

Using cpu device


c:\Users\OWNER\AppData\Local\Programs\Python\Python313\Lib\site-packages\gymnasium\spaces\box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
c:\Users\OWNER\AppData\Local\Programs\Python\Python313\Lib\site-packages\gymnasium\spaces\box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


-----------------------------
| time/              |      |
|    fps             | 31   |
|    iterations      | 1    |
|    time_elapsed    | 65   |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 27          |
|    iterations           | 2           |
|    time_elapsed         | 147         |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.011660212 |
|    clip_fraction        | 0.117       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -0.000334   |
|    learning_rate        | 0.0003      |
|    loss                 | 7.16e+03    |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0118     |
|    value_loss           | 1.71e+04    |
-----------------------------------------
----------------------------------

c:\Users\OWNER\AppData\Local\Programs\Python\Python313\Lib\site-packages\stable_baselines3\common\save_util.py:284: UserWarning: Path 'agents' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [8]:
# Save the model
model.save("./agents/western-agent-fixed2") # creates .zip in current working directory

In [9]:
loaded_model = PPO.load("./agents/western-agent-various-rates-3")

In [10]:
# Create vec env
env = SumoEnv(use_gui=False, use_random=True, use_actions=True, spawn_rate=0.50)
env.step(0)
# check_env(env)
env = DummyVecEnv([lambda: env])

In [11]:
# Run test episodes
num_test_episodes = 25
all_scores = []
wait_log = []
emissions_log = []

total_congestion_avg = 0
congestion_avg_count = 0
    
total_speed_avg = 0
speed_avg_count = 0

# Run multiple test episodes
for episode in range(num_test_episodes):
    obs = env.reset()
    done = False
    episode_score = 0
    info = {}
    while not done:
        action, _states = loaded_model.predict(obs, deterministic=True)  # Use deterministic actions during testing
        obs, reward, done, info = env.step(action)
        episode_score += reward

    all_scores.append(episode_score)

    # print(info)
    emissions_log.append(info[0].get("emissions"))
    wait_times = info[0].get('vehicle_wait_log', {}).values()
    total_wait_time = sum(wait_times)
    num_cars = len(wait_times)
    episode_mean_wait = total_wait_time / num_cars
    wait_log.append(episode_mean_wait)
    
    # Extract congestion and speed metrics    
    congestion_value = info[0].get('total_congestion_avg')
    speed_value = info[0].get('total_speed_avg')

    if congestion_value is not None:
        total_congestion_avg += congestion_value
        congestion_avg_count += 1
            
    if speed_value is not None:
        total_speed_avg += speed_value
        speed_avg_count += 1
        
traci.close()

mean_score = np.mean(all_scores)
mean_wait_time = np.mean(wait_log)
mean_emissions = np.mean(emissions_log) if emissions_log else None
mean_congestion = total_congestion_avg / congestion_avg_count if congestion_avg_count > 0 else 0
mean_speed = total_speed_avg / speed_avg_count if speed_avg_count > 0 else 0


print(f"Mean Score over {num_test_episodes} episodes: {mean_score}")
print(f"Mean wait time over {num_test_episodes} episodes: {mean_wait_time}")
print(f"Mean congestion over {num_test_episodes} episodes: {mean_congestion}")
print(f"Mean speed over {num_test_episodes} episodes: {mean_speed}")
print(f"Mean emissions over {num_test_episodes} episodes: {mean_emissions} milligrams (mg)")
print(wait_log)

Mean Score over 25 episodes: -510.4653015136719
Mean wait time over 25 episodes: 5.1092
Mean congestion over 25 episodes: 1.3056036126667385
Mean speed over 25 episodes: 6.348424704082825
Mean emissions over 25 episodes: 2589.141686369208 milligrams (mg)
[5.65, 5.38, 3.58, 4.63, 4.08, 3.93, 4.72, 8.44, 5.48, 4.96, 3.46, 4.42, 5.64, 5.61, 4.84, 5.76, 4.75, 6.11, 6.0, 6.19, 4.15, 5.91, 5.03, 5.33, 3.68]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

spawn_rates = np.arange(0.05, 0.85, 0.025)  # Adjust range as needed

# Initialize logs
mean_wait_times_rl = []
mean_emissions_rl = []
mean_wait_times_random = []
mean_emissions_random = []

num_test_episodes = 100  # Keep both cases consistent

for spawn_rate in spawn_rates:
    print("Beginning spawn rate =", spawn_rate)
    # ============================== RL Agent ====================================
    env_rl = SumoEnv(use_gui=False, use_random=True, use_actions=True, spawn_rate=spawn_rate)
    env_rl.step(0)
    env_rl = DummyVecEnv([lambda: env_rl])

    all_wait_times_rl = []  # Collect mean wait time for each episode
    all_emissions_rl = []   # Collect emissions for each episode

    for episode in range(num_test_episodes):
        obs = env_rl.reset()
        done = False
        info = {}

        while not done:
            action, _states = loaded_model.predict(obs, deterministic=True)
            obs, reward, done, info = env_rl.step(action)

        # Collect RL agent data
        if "emissions" in info[0]:
            all_emissions_rl.append(info[0]["emissions"])
        if "vehicle_wait_log" in info[0]:
            wait_times = list(info[0]["vehicle_wait_log"].values())
            episode_mean_wait = np.mean(wait_times) if wait_times else 0  # Calculate mean for this episode
            all_wait_times_rl.append(episode_mean_wait)  # Store mean for this episode

    traci.close()

    print(all_wait_times_rl)
    
    # Store RL results
    mean_wait_time_rl = np.mean(all_wait_times_rl)  # Mean of episode means
    mean_emission_rl = np.mean(all_emissions_rl) if all_emissions_rl else 0

    print("RL: spawn_rate=", spawn_rate, "wait time mean=", mean_wait_time_rl)
    print("Wait log length RL:", len(all_wait_times_rl))
    mean_wait_times_rl.append(mean_wait_time_rl)
    mean_emissions_rl.append(mean_emission_rl)

    # ============================== Random Actions ====================================
    env_random = SumoEnv(use_gui=False, use_random=True, use_actions=False, spawn_rate=spawn_rate)

    all_wait_times_random = []  # Collect mean wait time for each episode
    all_emissions_random = []   # Collect emissions for each episode

    for episode in range(num_test_episodes):
        done = False
        info = {}

        while not done:
            action = env_random.action_space.sample()  # Random actions
            state, reward, done, truncated, info = env_random.step(action)

        # Collect Random actions data
        if "emissions" in info:
            all_emissions_random.append(info["emissions"])
        if "vehicle_wait_log" in info:
            wait_times = list(info["vehicle_wait_log"].values())
            episode_mean_wait = np.mean(wait_times) if wait_times else 0  # Calculate mean for this episode
            all_wait_times_random.append(episode_mean_wait)  # Store mean for this episode

    traci.close()

    # Store Random results
    mean_wait_time_random = np.mean(all_wait_times_random)  # Mean of episode means
    mean_emission_random = np.mean(all_emissions_random) if all_emissions_random else 0

    print("Random actions: spawn_rate=", spawn_rate, "wait time mean=", mean_wait_time_random)
    print("Random actions wait log length: ", len(all_wait_times_random))
    mean_wait_times_random.append(mean_wait_time_random)
    mean_emissions_random.append(mean_emission_random)

# ============================== Plot Results ====================================

# Define traffic density boundaries
low_traffic_end = 0.3
high_traffic_start = 0.6

# Colors for different traffic densities
low_color = "lightblue"
medium_color = "lightgreen"
high_color = "lightcoral"

# ========================= Mean Wait Time Plot =========================
plt.figure(figsize=(8, 5))

plt.axvspan(min(spawn_rates), low_traffic_end, color=low_color, alpha=0.2)
plt.axvspan(low_traffic_end, high_traffic_start, color=medium_color, alpha=0.2)
plt.axvspan(high_traffic_start, max(spawn_rates), color=high_color, alpha=0.2)


plt.plot(spawn_rates, mean_wait_times_rl, marker="o", color="red", label="RL Agent")
plt.plot(spawn_rates, mean_wait_times_random, marker="s", color="blue", linestyle="dashed", label="Traditional System")

plt.xlabel("Vehicle Spawn Rate")
plt.ylabel("Mean Wait Time (s)")
plt.title("Effect of Traffic Density on Mean Wait Time")
plt.grid(True)
plt.legend()

# Add text labels for traffic regions
#plt.text(0.05, max(mean_wait_times_rl) * 0.9, "Low Traffic", fontsize=12, color="black")
#plt.text(0.35, max(mean_wait_times_rl) * 0.9, "Medium Traffic", fontsize=12, color="black")
#plt.text(0.65, max(mean_wait_times_rl) * 0.9, "High Traffic", fontsize=12, color="black")

plt.savefig("mean_wait_time_plot_final2.png")
plt.show()

# ========================= Mean Emissions Plot =========================
plt.figure(figsize=(8, 5))

plt.axvspan(min(spawn_rates), low_traffic_end, color=low_color, alpha=0.2)
plt.axvspan(low_traffic_end, high_traffic_start, color=medium_color, alpha=0.2)
plt.axvspan(high_traffic_start, max(spawn_rates), color=high_color, alpha=0.2)


plt.plot(spawn_rates, mean_emissions_rl, marker="o", color="red", label="RL Agent")
plt.plot(spawn_rates, mean_emissions_random, marker="s", color="blue", linestyle="dashed", label="Traditional System")

plt.xlabel("Vehicle Spawn Rate")
plt.ylabel("Mean Emissions (mg)")
plt.title("Effect of Traffic Density on Emissions")
plt.grid(True)
plt.legend()

plt.savefig("mean_emissions_plot_final2.png")
plt.show()

Beginning spawn rate = 0.05


c:\Users\OWNER\AppData\Local\Programs\Python\Python313\Lib\site-packages\gymnasium\spaces\box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
c:\Users\OWNER\AppData\Local\Programs\Python\Python313\Lib\site-packages\gymnasium\spaces\box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


[np.float64(4.65), np.float64(4.434343434343434), np.float64(3.8585858585858586), np.float64(4.06), np.float64(4.252525252525253), np.float64(4.86), np.float64(5.181818181818182), np.float64(5.606060606060606), np.float64(3.61), np.float64(3.5656565656565657), np.float64(4.707070707070707), np.float64(4.55), np.float64(3.5656565656565657), np.float64(3.787878787878788), np.float64(4.404040404040404), np.float64(4.444444444444445), np.float64(3.919191919191919), np.float64(3.98989898989899), np.float64(4.11), np.float64(4.474747474747475), np.float64(3.9), np.float64(4.373737373737374), np.float64(3.88), np.float64(4.929292929292929), np.float64(3.9696969696969697), np.float64(4.575757575757576), np.float64(4.09), np.float64(4.0), np.float64(3.5656565656565657), np.float64(5.77), np.float64(4.626262626262626), np.float64(3.606060606060606), np.float64(5.57), np.float64(4.224489795918367), np.float64(4.505050505050505), np.float64(5.303030303030303), np.float64(2.9696969696969697), np.fl